<a href="https://colab.research.google.com/github/niv11212/niv11212/blob/main/Brazilian_E_Commerce_Public_Dataset_by_Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hypothesis (1): Products with few images will sell less than products with many images.

In [36]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from sklearn import linear_model







In [37]:
#Creating the dataframe
order_items_dataset = "/content/olist_order_items_dataset.csv"
orderr_items_df = pd.read_csv(order_items_dataset)
products_dataset = "/content/olist_products_dataset.csv"
products_df = pd.read_csv(products_dataset)

order_items_df_t = orderr_items_df.groupby(by = ['product_id']).size().reset_index() #group and count proucts
order_items_df_t['count_sales'] = order_items_df_t[0] 
order_items_df_t = order_items_df_t.drop(columns=0)
order_items_df_t = order_items_df_t.sort_values(by = ['count_sales'])
order_items_df = pd.merge(order_items_df_t, orderr_items_df[['price','product_id','freight_value']],on='product_id') #join dfs
order_items_df = order_items_df.drop_duplicates()

joined_df = pd.merge(order_items_df[['price','product_id','count_sales','freight_value']], products_df[['product_width_cm','product_height_cm', 'product_length_cm',                                                                          'product_weight_g','product_id','product_photos_qty' ]],on='product_id')
joined_df = joined_df[joined_df['price']<100] #removing outliers
joined_df=joined_df.dropna().reset_index(drop=True)
joined_df1 = joined_df.drop_duplicates(subset=['product_id']) 
joined_df1


,price,product_id,count_sales,freight_value,product_width_cm,product_height_cm,product_length_cm,product_weight_g,product_photos_qty
0,59.9,911e26785862f450cf2240fc5168f83d,1,23.35,15.0,30.0,25.0,300.0,4.0
1,47.9,911cfb37438b7fa9ad54535990178868,1,14.52,15.0,15.0,16.0,500.0,4.0
2,99.9,911b8d269dbaa6c005b23d04e1c7fcbc,1,33.15,15.0,15.0,60.0,700.0,2.0
3,19.9,9117307cf75efa6e964d697f0d1297f9,1,24.84,11.0,9.0,16.0,200.0,5.0
4,44.9,9116ce8acab686803dfd443f5963e69c,1,7.39,32.0,4.0,34.0,300.0,3.0
...,...,...,...,...,...,...,...,...,...
44112,49.9,368c6c730842d78016ad823897a372db,388,13.37,30.0,22.0,30.0,1650.0,2.0
44173,49.9,389d119b48cf3043d311335e499d9c6b,392,17.60,40.0,22.0,37.0,1750.0,2.0
44237,53.9,422879e10f46682990de24d770e7f83d,484,0.00,30.0,22.0,30.0,1550.0,2.0
44306,74.0,99a4788cb24856965c36a24e339b6058,488,23.32,40.0,10.0,50.0,1383.0,1.0


In [38]:


fig = px.pie(products_df[products_df['product_photos_qty']<9], values='product_photos_qty', names='product_photos_qty', title='Types of products in terms of the amount of images in the product')
fig.show()

fig = px.pie(joined_df1[joined_df1['product_photos_qty']<9], values='count_sales', names='product_photos_qty', title='Types of sold products in terms of the amount of images in the product')
fig.show()





#Hypothesis (1) conclusion:
* The hypothesis was disproved, the absolute majority of the products that are sold have one or two images.
*  This does not mean that there is no effect of the number of images on sales because there can be other explanations such as - products with one image are cheap products and therefore they are sold more.
*  More research is needed.

#Hypothesis (2):
#Some days of the week have fewer customer orders 

In [39]:
order_dataset = "/content/olist_orders_dataset.csv"
orders_df = pd.read_csv(order_dataset) #Creating the dataframe
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp']) #changing the type of the column to timestamp
orders_df['weekday'] = orders_df['order_purchase_timestamp'].dt.dayofweek #The day of the week with Monday=0, Sunday=6.
orders_df_t = orders_df.groupby(by = ['weekday']).size().reset_index() #How many sales are there on each day throughout the database
orders_df_t['count_orders'] = orders_df_t[0] 
orders_df = orders_df_t.drop(columns=0)

orders_df

,weekday,count_orders
0,0,16196
1,1,15963
2,2,15552
3,3,14761
4,4,14122
5,5,10887
6,6,11960


In [40]:


colors = ['grey',] * 7
colors[0] = 'blue'
colors[5] = 'crimson'


fig = go.Figure(data=[go.Bar(
    x= orders_df['weekday'],
    y= orders_df['count_orders'],
    marker_color=colors 
)])
fig.update_layout(title_text='Sales on each day')
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4, 5, 6],
        ticktext = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    )
)

fig.show()

#Hypothesis (2) conclusion:
* There is a difference of one and a half times between the least busy day of the week (Saturday) and the busiest day (Monday).
* I would recommend adding shifts for employees on the busy days and reducing shifts for employees from the less busy days - if this step has not been taken so far.

#Hypothesis (3):
* There are significant differences between the product categories in terms of customer experience.

In [41]:
#Creating the dataframe from 4 datasets
category_name_dataset = "/content/product_category_name_translation.csv"
category_name_df = pd.read_csv(category_name_dataset)
order_reviews_dataset = "/content/olist_order_reviews_dataset.csv"
order_reviews_df = pd.read_csv(order_reviews_dataset)
products_df_E = pd.merge(products_df,category_name_df,on='product_category_name',how='left')
products_df_E = products_df_E[['product_id', 'product_category_name_english']]
joined_df1 = pd.merge(orderr_items_df[['product_id','order_id']], products_df_E[['product_category_name_english','product_id']],on='product_id')
order_reviews_df = order_reviews_df[['order_id','review_score']]
joined_df2 = pd.merge(joined_df1, order_reviews_df,on='order_id')

joined_df2 = joined_df2.groupby(by = ['product_category_name_english']).mean().reset_index() #Finding the average value for each category

#Finding the 4 best categories
low_rating_category = joined_df2.sort_values('review_score',ascending=True).reset_index()
low_rating_category = low_rating_category.drop(columns='index')
low_rating_category = low_rating_category.head(4)

#Finding the 4 worst categories
top_rating_category = joined_df2.sort_values('review_score',ascending=False).reset_index()
top_rating_category = top_rating_category.drop(columns='index')
top_rating_category = top_rating_category.head(4)

#Union between the tables to display them in one graph
mixed_rating_category = pd.concat([low_rating_category,top_rating_category]).reset_index() #Union
mixed_rating_category = mixed_rating_category.drop(columns='index')
mixed_rating_category

,product_category_name_english,review_score
0,security_and_services,2.500000
1,diapers_and_hygiene,3.256410
2,office_furniture,3.493183
3,home_comfort_2,3.629630
4,cds_dvds_musicals,4.642857
5,fashion_childrens_clothes,4.500000
6,books_general_interest,4.446266
7,costruction_tools_tools,4.444444


In [42]:

colors = ['blue',] * 8
colors[0] = 'crimson'
colors[1] = 'crimson'
colors[2] = 'crimson'
colors[3] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x= mixed_rating_category['product_category_name_english'],
    y= mixed_rating_category['review_score'],
    marker_color=colors 
)])
fig.update_layout(title_text='4 categories that received the lowest rating and 4 that received the highest rating')


#Hypothesis (3) conclusion:
When comparing the four categories with the highest rating to the four with the lowest rating - significant differences are revealed. Further research will be- to find what is common to the good reviews and what is common to the bad reviews from each category. This way the company will be able to preserve the categories with the highest rating and improve the categories with the lowest rating.

#Unfinished addition:
an attempt to build a multivariable regression with which it will be possible to predict the shipping cost. Continued work will be:
* Adding variables
* Significance test for each variable separately

In [43]:
joined_df['volume'] = joined_df['product_width_cm']*joined_df['product_height_cm']*joined_df['product_length_cm'] #creating a volume column

features = ['price', 'volume','product_photos_qty']
target = 'freight_value'

X = joined_df[features].values.reshape(-1, len(features))
y = joined_df[target].values

ols = linear_model.LinearRegression()
model = ols.fit(X, y)
price_coef = model.coef_[0]
volume_coef = model.coef_[1]
photos_coef = model.coef_[2]
intercept =  model.intercept_

print(price_coef)
print(volume_coef)
print(photos_coef)
print(intercept)

0.048387586543821665
0.00017760969858672247
0.012139191529272024
12.523310334783798
